<a href="https://colab.research.google.com/github/CodeByAtharva/Agentic-AI-Internship/blob/master/day%208/travel_assistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -q langchain-google-genai langchain-core tavily-python requests


In [27]:
# AI Travel Assistant with Gemini, OpenWeatherMap & Tavily
# Designed for Google Colab

# Step 1: Install required packages
!pip install -q langchain-google-genai tavily-python requests

# Step 2: Import libraries
import requests
from tavily import TavilyClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from google.colab import userdata

# Step 3: Set up API keys
print("🔑 Setting up API Keys...")

GEMINI_API_KEY = userdata.get('GeminiApi')
OPENWEATHER_API_KEY = userdata.get('whether')
TAVILY_API_KEY = userdata.get('Tavily_api')

print("✅ API Keys configured!\n")

# Step 4: Initialize APIs
def get_weather(city, api_key):
    """Fetch real-time weather data from OpenWeatherMap"""
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        weather_summary = {
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
        return weather_summary
    except Exception as e:
        return {"error": str(e)}

def search_tavily(query, api_key, max_results=5):
    """Search for travel information using Tavily"""
    try:
        client = TavilyClient(api_key=TAVILY_API_KEY)
        response = client.search(query=query, max_results=max_results)
        return response.get('results', [])
    except Exception as e:
        return {"error": str(e)}

def generate_itinerary(city, duration, interests, time_preference, weather_data, search_results, api_key):
    """Generate personalized travel itinerary using Gemini LLM"""
    try:
        # Initialize Gemini model
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite",
            temperature=0.7,
            google_api_key=GEMINI_API_KEY
        )

        # Prepare context
        weather_info = f"""
Weather in {city}:
- Temperature: {weather_data.get('temperature', 'N/A')}°C (Feels like {weather_data.get('feels_like', 'N/A')}°C)
- Conditions: {weather_data.get('description', 'N/A')}
- Humidity: {weather_data.get('humidity', 'N/A')}%
- Wind Speed: {weather_data.get('wind_speed', 'N/A')} m/s
"""

        search_context = "\n\nRecommendations from search:\n"
        for idx, result in enumerate(search_results[:10], 1):
            search_context += f"{idx}. {result.get('title', 'N/A')}\n   {result.get('content', 'N/A')[:200]}...\n   URL: {result.get('url', 'N/A')}\n\n"

        # Create prompt
        system_prompt = SystemMessage(content="""You are an expert travel planner. Create detailed,
personalized day-wise travel itineraries based on the user's preferences, current weather conditions,
and local recommendations. Be specific with timings, locations, and practical travel tips.""")

        user_prompt = HumanMessage(content=f"""
Create a personalized {duration}-day travel itinerary for {city}.

User Preferences:
- Interests: {', '.join(interests)}
- Preferred time of day for activities: {time_preference}

{weather_info}
{search_context}

Please provide:
1. A brief weather summary and what to pack
2. Top 5 must-visit attractions (with brief descriptions)
3. 5 recommended hotels (range of budgets)
4. 5 recommended restaurants (variety of cuisines)
5. A detailed day-by-day itinerary with:
   - Morning, afternoon, and evening activities
   - Specific timings
   - Transportation tips
   - Estimated costs where possible
   - Weather-appropriate suggestions

Format the response in a clear, organized manner with headers and bullet points.
""")

        # Generate response
        response = llm.invoke([system_prompt, user_prompt])
        return response.content
    except Exception as e:
        return f"Error generating itinerary: {str(e)}"

# Step 5: Main Travel Assistant Function
def travel_assistant(city, duration, interests, time_preference):
    """Main function to orchestrate the travel planning"""

    print(f"\n{'='*60}")
    print(f"🌍 AI TRAVEL ASSISTANT")
    print(f"{'='*60}\n")

    print(f"📍 Destination: {city}")
    print(f"📅 Duration: {duration} days")
    print(f"🎯 Interests: {', '.join(interests)}")
    print(f"🕐 Preferred time: {time_preference}\n")

    # Step 1: Fetch Weather
    print("☁️ Fetching weather data...")
    weather_data = get_weather(city, OPENWEATHER_API_KEY)
    if "error" in weather_data:
        print(f"⚠️ Weather Error: {weather_data['error']}")
    else:
        print(f"✅ Weather: {weather_data['temperature']}°C, {weather_data['description']}\n")

    # Step 2: Search for recommendations
    print("🔍 Searching for attractions, hotels, and restaurants...")
    search_queries = [
        f"best attractions in {city}",
        f"top hotels in {city}",
        f"best restaurants in {city}",
        f"{city} travel guide {' '.join(interests)}"
    ]

    all_results = []
    for query in search_queries:
        results = search_tavily(query, TAVILY_API_KEY, max_results=3)
        if isinstance(results, list):
            all_results.extend(results)

    print(f"✅ Found {len(all_results)} recommendations\n")

    # Step 3: Generate itinerary with Gemini
    print("🤖 Generating personalized itinerary with Gemini AI...\n")
    itinerary = generate_itinerary(
        city, duration, interests, time_preference,
        weather_data, all_results, GEMINI_API_KEY
    )

    print(f"\n{'='*60}")
    print(f"📋 YOUR PERSONALIZED TRAVEL ITINERARY")
    print(f"{'='*60}\n")
    print(itinerary)
    print(f"\n{'='*60}")
    print(f"✨ Happy travels! ✨")
    print(f"{'='*60}\n")

    return {
        "city": city,
        "duration": duration,
        "interests": interests,
        "time_preference": time_preference,
        "weather": weather_data,
        "itinerary": itinerary
    }

# Step 6: Interactive User Input
print("\n" + "="*60)
print("🌟 WELCOME TO AI TRAVEL ASSISTANT")
print("="*60 + "\n")

# Get user inputs
city = input("Enter destination city: ").strip()
duration = int(input("Enter trip duration (in days): ").strip())

print("\nSelect your interests (comma-separated):")
print("Examples: food, adventure, culture, history, nature, shopping, nightlife, art")
interests_input = input("Your interests: ").strip()
interests = [i.strip() for i in interests_input.split(",")]

print("\nPreferred time of day for activities:")
print("Options: morning, afternoon, evening, flexible")
time_preference = input("Your preference: ").strip()

# Generate the travel plan
result = travel_assistant(city, duration, interests, time_preference)

print("\n🎉 Thank you for using AI Travel Assistant!")

🔑 Setting up API Keys...
✅ API Keys configured!


🌟 WELCOME TO AI TRAVEL ASSISTANT

Enter destination city: Pune
Enter trip duration (in days): 5

Select your interests (comma-separated):
Examples: food, adventure, culture, history, nature, shopping, nightlife, art
Your interests: Forts

Preferred time of day for activities:
Options: morning, afternoon, evening, flexible
Your preference: morning

🌍 AI TRAVEL ASSISTANT

📍 Destination: Pune
📅 Duration: 5 days
🎯 Interests: Forts
🕐 Preferred time: morning

☁️ Fetching weather data...
✅ Weather: 26.34°C, clear sky

🔍 Searching for attractions, hotels, and restaurants...
✅ Found 12 recommendations

🤖 Generating personalized itinerary with Gemini AI...


📋 YOUR PERSONALIZED TRAVEL ITINERARY

Pune, with its historical significance and pleasant weather, offers a delightful experience, especially for fort enthusiasts. Here's a personalized 5-day itinerary designed to maximize your mornings and immerse you in the grandeur of its forts and histori